In [37]:
import jax
from jax import jit
from jax.lax import fori_loop as forloop
import jax.numpy as np
import numpy as onp
from math import prod
from sklearn.decomposition import NMF

In [36]:
def nmf(a,u=None,v=None,rank=None,iterations=None):
    
    slices = slice(None)
    
    def nndsvd(a,u,v):
        def func(i,x):
            s,u,v = x
            
            z,x,y = s[i],u[slices,i],v[i,slices]
            
            x_positive,y_positive = absolute(maximums(x,0)),absolute(maximums(y,0))
            x_negative,y_negative = absolute(minimums(x,0)),absolute(minimums(y,0))
            
            
            n,m = sqrt(dot(x,x)),sqrt(dot(y,y))
            
            
#             W = np.zeros_like(U)
#             H = np.zeros_like(V)

#             # The leading singular triplet is non-negative
#             # so it can be used as is for initialization.
#             W[:, 0] = np.sqrt(S[0]) * np.abs(U[:, 0])
#             H[0, :] = np.sqrt(S[0]) * np.abs(V[0, :])

            for j in range(1, n_components):
#                 x, y = U[:, j], V[j, :]

                # extract positive and negative parts of column vectors
                x_p, y_p = np.maximum(x, 0), np.maximum(y, 0)
                x_n, y_n = np.abs(np.minimum(x, 0)), np.abs(np.minimum(y, 0))

                # and their norms
                x_p_nrm, y_p_nrm = norm(x_p), norm(y_p)
                x_n_nrm, y_n_nrm = norm(x_n), norm(y_n)

                m_p, m_n = x_p_nrm * y_p_nrm, x_n_nrm * y_n_nrm

                # choose update
                if m_p > m_n:
                    u = x_p / x_p_nrm
                    v = y_p / y_p_nrm
                    sigma = m_p
                else:
                    u = x_n / x_n_nrm
                    v = y_n / y_n_nrm
                    sigma = m_n

                lbd = np.sqrt(S[j] * sigma)
                W[:, j] = lbd * u
                H[j, :] = lbd * v            

            u = inplace(u,(slices,i),sqrt(z)*x)
            v = inplace(v,(i,slices),sqrt(z)*y)
            
        
            x = s,u,v
            return x
        
        s,u,v = np.linalg.svd(a)
        
        i = 0
        u = inplace(u,(slices,i),sqrt(s[i])*abs(u[slices,i]))
        v = inplace(v,(i,slices),sqrt(s[i])*abs(v[i,slices]))
                                  
        start,end,x = 1,rank,(s,u,v)
        x = forloop(start,end,func,x)
        s,u,v = x
        
        return s,u,v
    
    def dsvd(u,a,v):
        x,y = add(u,0),add(v,1)
        s,u,v = x,y,u*1/x,transpose(transpose(v)*1/y)
        return s,u,v
                               
    def init(a,u=None,v=None):
        if u is None or v is None:
            s,u,v = nndsvd(a)
        else:
            s,u,v = dsvd(a,u,v)
        
        return s,u,v
                               
    rank = a.shape[-1] if rank is None else rank
    s,u,v = init(a,u=u,v=v)
            
    return
    
def nvd(a,**kwargs):
    print(k)
    u,v,s = nmf(**kwargs)
    t,q = add(u,0),add(v,1)
    u,v,s = u*1/t,transpose(transpose(v)*1/q),t*q
    return dot(u*s,v)

def svd(a):
    return 

def inplace(obj,index,item,op='set'):
    obj = getattr(obj.at[index],op)(item)
                               
def diag(a):
    return np.diag(a)

def add(a,axis=None):
    return a.sum(axis=axis)

def reshape(a,shape):
    return np.reshape(a,shape)

def transpose(a):
    return a.transpose()

def conjugate(a):
    return a.conjugate()

def dagger(a):
    return conjugate(transpose(a))

def dot(a,b=None):
    return np.dot(a,b if b is not None else a)

def norm(a,b=None):
    return sqrt(dot(a,b))

def sqrt(a):
    return np.sqrt(a)

def absolute(a):
    return np.abs(a)

def maximums(a,b):
    return np.maximum(a,b)

def minimums(a,b):
    return np.minimum(a,b)

def init(*shape,**kwargs):
    return (1/sqrt(prod(shape)))*np.ones(shape=shape,dtype=dtype)
n = 4
q = 2
d = q**n
i = 1
k = int((q**(i+1))*(1/2))
dtype = 'float'

size = d
shape = (q**(i+1),q**(n-i-1))

a = dot(reshape(init(size,dtype=dtype),shape))

b = nvd(a,k=k)

print(a)
print(b)

IndentationError: expected an indented block (2454233856.py, line 5)

In [23]:
a = np.arange(6).reshape(2,3)
print(a)
print(a.sum(0))

[[0 1 2]
 [3 4 5]]
[3 5 7]
